# 02 — Definición de KPIs

Diseño participativo de indicadores con operarios y administración.

**Principios**:
- Accionables (¿qué hacer si el KPI está en rojo?)
- Medibles con datos disponibles
- Relevantes para el negocio salinero

In [ ]:
import polars as pl

# Cargar datos procesados
df = pl.read_parquet("../data/processed/produccion.parquet")

# KPI 1: Rendimiento (tn/ha)
rendimiento = df.select([
    pl.col("fecha"),
    (pl.col("toneladas") / (pl.col("area_pileta") / 10000)).alias("rendimiento")
])

# KPI 2: Eficiencia energética (kWh/tn)
eficiencia = df.select([
    pl.col("fecha"),
    (pl.col("consumo_kwh") / pl.col("toneladas")).alias("kwh_tn")
])

# KPI 3: Calidad premium (% refinada)
calidad = df.group_by("fecha").agg(
    (pl.sum("toneladas").filter(pl.col("calidad") == "refinada") / pl.sum("toneladas") * 100).alias("refinada_pct")
)

In [ ]:
# Umbrales propuestos (validados con operarios)
UMBRALES = {
    "rendimiento_min": 15.0,   # tn/ha (mínimo aceptable)
    "kwh_max": 28.0,          # kWh/tn (máximo eficiente)
    "refinada_min": 60.0      # % (mínimo para mercado premium)
}

# Visualización de umbrales
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.hist(rendimiento["rendimiento"], bins=20, alpha=0.7)
plt.axvline(UMBRALES["rendimiento_min"], color='r', linestyle='--')
plt.title("Rendimiento (tn/ha)")

plt.subplot(1, 3, 2)
plt.hist(eficiencia["kwh_tn"], bins=20, alpha=0.7, color='orange')
plt.axvline(UMBRALES["kwh_max"], color='r', linestyle='--')
plt.title("kWh/tn")

plt.subplot(1, 3, 3)
plt.hist(calidad["refinada_pct"], bins=20, alpha=0.7, color='green')
plt.axvline(UMBRALES["refinada_min"], color='r', linestyle='--')
plt.title("% Refinada")

plt.tight_layout()
plt.show()

## KPIs aprobados por la cooperativa

| KPI | Fórmula | Umbral verde | Umbral rojo | Acción si rojo |
|-----|---------|--------------|-------------|----------------|
| **Rendimiento** | tn / ha | ≥ 18.0 | < 15.0 | Revisar evaporación, limpieza de piletas |
| **Eficiencia energética** | kWh / tn | ≤ 25.0 | > 28.0 | Verificar bombas, ajustar presión |
| **Calidad premium** | % refinada | ≥ 65% | < 60% | Ajustar tiempos de lavado/secado |
| **Trazabilidad** | % con lote SENASA | 100% | < 100% | Detener despacho, registrar lotes |

➡️ Siguiente paso: modelo de pronóstico en `03_forecast_model.ipynb`